# Import libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import tabulate
import sys

# Importing CSV Data

In [2]:
allCik = pd.read_csv("ALL_CIK.csv", low_memory= False)

# Check for valid CIK

In [3]:
#function to check if entered CIK is valid

def CheckandAcceptCIK():
    flag = False
    num= float(input("Please enter your CIK number: "))
    
    for i in allCik["CIK"]:
        if (num == i):
            flag = True 
            
    if(flag == False):
        print("CIK is wrong")
        CheckandAcceptCIK()
    else:
        result_string = str(num)
        return result_string[0: (len(result_string) -2 )]

# Add Leading Zeros

In [4]:
#function to add leading zeroes 
length_cik = 0;
cik = '';
def calc_cik(cik):
    length_cik = len(cik)
    final_length = 10 - length_cik
    i=0
    while i < final_length:
        cik ='0' + cik
        i= i+1
        
    return cik


# Calculate Financial Year Part

In [7]:
#function to calculate financial year part which is added to our link
def calc_year():
    year = int(input("Enter the Financial year in 20XX format (current or lesser): "))
    import datetime
    now = datetime.datetime.now()
    if (year < 1990 or year> now.year):
        print('Please Enter your year as mentioned earlier')
        calc_year()
    else:
        year_str = str(year)
        return year_str[-2:]

# Generating the URL

In [6]:
cik = CheckandAcceptCIK()
zero_cik = calc_cik(cik)
year = calc_year()
website = "https://www.sec.gov/Archives/edgar/data/" + cik+"/" + zero_cik +"/" + zero_cik+ "-" + year +"-000007-index.htm"
website

Please enter your CIK number: 51143
Enter the Financial year in 20XX format (current or lesser): 2013


'https://www.sec.gov/Archives/edgar/data/51143/0000051143/0000051143-13-000007-index.htm'

# WebScrapping Tables from Generated URL 

In [8]:
res = requests.get(website)
soup = BeautifulSoup(res.content,'lxml')
#tables = soup.find('table')
tbl = soup.findAll('table')


In [9]:
print(len(tbl))

2


In [11]:
for table in tbl:
    for tr in table.findAll('tr'):
        for td in tr.findAll('td')[:2]:
            filing_type = td.string
            if filing_type=='10-Q':
                gen_links = [a['href'] for a in tr.findAll('a')]
                print('generated-link is:   ',gen_links)
                for links in gen_links:
                    web_link = 'https://www.sec.gov' + links
                    print('web-link is:   ',web_link)
                    res_new = requests.get(web_link)
                    soup_new = BeautifulSoup(res_new.content,'lxml')
                    tables_new = soup_new.find_all('table', attrs={'border':"1", 'cellpadding':"0", 'cellspacing':"0", 'style':"border:none;border-collapse:collapse;width:100%;"})
                    i = 1
                    print('length of table is:',len(tables_new))
                    for new_tab in tables_new:
                        df = pd.read_html(str(new_tab))
                        df[0].to_csv('my_data'+ str(i) + '.csv')
                        i = i + 1

generated-link is:    ['/Archives/edgar/data/51143/000005114313000007/ibm13q3_10q.htm']
web-link is:    https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/ibm13q3_10q.htm
length of table is: 106
